In [55]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
import importlib
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# Feature Engineering for net new features

In [62]:
# --------- #
# LEAVE ME  #
# --------- #
ticker = 'QQQ'
include_minute_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]
df_daily, feature_sets, return_cols, daily_cols, feature_dict, features = train_flow.import_data(ticker, include_minute_feats, returns)

# ---------- #
# UPDATE ME  #
# ---------- #
# Add any new features
df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
cutoff_date = '2026-02-06' # Unlessretraining all baseline models, leave alone

# --------- #
# LEAVE ME  #
# --------- #
df_main = df_daily[df_daily['Date'] <= cutoff_date].copy() #cutoff date for baseline performance metrics

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope'])


# Run Train_Flow

In [ ]:
import train_flow
import importlib
importlib.reload(train_flow)

# --------- #
# LEAVE ME  #
# --------- #
# Handling of new features and permutation importance
pi_handlings = ['exclude_new', 'include_new', 'run_separately']
train_flow.pi_handling_test(pi_handlings)

"""
Applicable when appending new features to an existing set
run_separately: runs pi for old and new feature set separately --> combines the top features from each set at end
exclude_new: runs pi only for old feature set then appends all new features onto the trimmed list --> combines the top old list features with all new features 
include_new: runs pi for combined old and new feature set --> if new features aren't adding incremental value they will be completely excluded
"""


# ---------- #
# UPDATE ME  #
# ---------- #
new_features = feature_dict['experimental_slope']
target_horizons = [2]#, 10, 20, 30]
train_years = [5]
pi_years = [1.5]
min_feats = [8]
days_assessed = 120
test_day = 1
feature_cols = feature_dict['ma_num'] #feature_cols
list_name = "ma_num+exp"


# --------- #
# LEAVE ME  #
# --------- # 
models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
results_df, perm_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                   train_years, pi_years, min_feats, list_name, new_features)

Running for horizon 2 | exclude_new
24 | 20 | ['CCI_14', 'CCI_5', 'Close_slope10', 'Close_slope25', 'Close_slope50', 'Max_10_Rows_Since', 'Min_10_Rows_Since', 'Min_120_Rows_Since', 'Min_240_Rows_Since', 'Min_30_Rows_Since', 'Min_60_Rows_Since', 'Williams_%R_14', 'Williams_%R_5', 'Zscore_10', 'Zscore_25', 'Zscore_5', 'Zscore_50', 'num_days_100', 'num_days_200', 'num_days_50']
Run 1/120 | Train: 2021-03-17 → 2026-01-30 | Test: 2026-02-04 → 2026-02-04 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
24 | 19 | ['CCI_14', 'CCI_5', 'Close_slope10', 'Close_slope25', 'Close_slope50', 'Max_10_Rows_Since', 'Min_10_Rows_Since', 'Min_120_Rows_Since', 'Min_240_Rows_Since', 'Min_30_Rows_Since', 'Min_60_Rows_Since', 'Williams_%R_14', 'Williams_%R_5', 'Zscore_10', 'Zscore_25', 'Zscore_5', 'Zscore_50', 'num_days_100', 'num_days_50']
Run 2/120 | Train: 2021-03-16 → 2026-01-29 | Test: 2026-02-03 → 2026-02-03 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
24 | 19 | ['CCI_14', 'CCI_5', 'Close

# Run for combo of sets

In [ ]:
import train_flow
import importlib
importlib.reload(train_flow)
from itertools import combinations


# --------- #
# LEAVE ME  #
# --------- #
# Handling of new features and permutation importance
pi_handlings = ['include_new'] #'exclude_new', 'include_new', 'run_separately'
train_flow.pi_handling_test(pi_handlings)
keys = list(feature_dict.keys())
two_combos = list(combinations(keys, 2))

"""
Applicable when appending new features to an existing set
run_separately: runs pi for old and new feature set separately --> combines the top features from each set at end
exclude_new: runs pi only for old feature set then appends all new features onto the trimmed list --> combines the top old list features with all new features 
include_new: runs pi for combined old and new feature set --> if new features aren't adding incremental value they will be completely excluded
"""


for a, b in two_combos:
    # ---------- #
    # UPDATE ME  #
    # ---------- #
    new_features = feature_dict[b]
    target_horizons = [2]#, 10, 20, 30]
    train_years = [5]
    pi_years = [1.5]
    min_feats = [8]
    days_assessed = 120
    test_day = 1
    feature_cols = feature_dict[a] #feature_cols
    list_name = f"{a}-{b}"
    print(f"{a}-{b}")


    # --------- #
    # LEAVE ME  #
    # --------- # 
    models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
    results_df, perm_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                    train_years, pi_years, min_feats, list_name, new_features)

ma_lag-ma_rel
Running for horizon 2 | include_new
87 | 8 | All Cols: ['Close_Rel_Max10', 'Close_Rel_Min25', 'Past_Return_10_sum10', 'Past_Return_20_sum10', 'Past_Return_2_sum10', 'Past_Return_5_sum10', 'Rel_Min_30', 'SMA_10_Lag200_min']
Run 1/120 | Train: 2021-03-17 → 2026-01-30 | Test: 2026-02-04 → 2026-02-04 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
87 | 8 | All Cols: ['Close_Rel_Max10', 'Close_Rel_Min25', 'Close_Rel_Min50', 'Past_Return_10_sum10', 'Past_Return_2_sum10', 'Rel_Min_30', 'SMA_10_Lag10_max', 'SMA_10_Lag200_min']
Run 2/120 | Train: 2021-03-16 → 2026-01-29 | Test: 2026-02-03 → 2026-02-03 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
87 | 8 | All Cols: ['Close_Rel_Max10', 'Close_Rel_Min25', 'Past_Return_10_sum10', 'Past_Return_20_sum10', 'Past_Return_2_sum10', 'Rel_Min_60', 'SMA_10_Lag10_max', 'SMA_10_Lag200_min']
Run 3/120 | Train: 2021-03-15 → 2026-01-28 | Test: 2026-02-02 → 2026-02-02 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
87 | 8 | A

# Save most recent run

In [110]:
# ---------- #
# UPDATE ME  #
# ---------- #
df = pd.read_csv("baseline_new.csv")


# --------- #
# LEAVE ME  #
# --------- #
cols = ['model', 'test_days', 'pred', 'acc', 'test_n', 'test_pos_n', 'train_n', 'test_start', 'test_end', 'train_years', 'feature_set', 'horizon',
        'pi_size', 'min_feats']
df_new = results_df[cols].copy()
df_concat = pd.concat([df[cols], df_new], ignore_index=True)


# ---------- #
# UPDATE ME  #
# ---------- #
df_concat.to_csv('baseline_new.csv', index=False)

# Run Performance_Flow

In [113]:
import performance_flow
importlib.reload(performance_flow)

# ---------- #
# UPDATE ME  #
# ---------- #
results_file_name = "baseline_new.csv" # Match prior cell saved as file name
min_th = 0.55
cov_th = 0.75


# --------- #
# LEAVE ME  #
# --------- #
return_cols, perf_df = performance_flow.import_data(results_file_name, df_daily)
composite_score = performance_flow.run_performance(perf_df, min_th, cov_th)
bucket_df = performance_flow.bucket_scores(df_daily, perf_df, returns, min_th)

In [ ]:
composite_score[
    (composite_score["horizon"] == 2) &
    (composite_score["feature_set"].str.contains("num|exp", regex=True))
].head(10)

,horizon,model,train_years,feature_set,pi_size,min_feats,pos_rate,records,bal_prec,bal_prec_|0.75|,brier,log_loss,mcc,pprec,nprec,cov_|0.75|,pprec_|0.75|,nprec_|0.75|,composite
131,2,xgboost-3,6,ma_num_ba,1.5,100,0.56,112.0,0.59,0.57,0.33,4.09,0.17,0.63,0.55,0.77,0.57,0.57,0.47
281,2,xgboost-3,6,experimental_slope_ba,1.5,6,0.54,114.0,0.58,0.56,0.33,3.48,0.15,0.59,0.57,0.82,0.60,0.52,0.46
21,2,xgboost-3,6,ma_num-baseline,1.5,8,0.56,222.0,0.58,0.55,0.34,3.85,0.15,0.62,0.53,0.78,0.56,0.55,0.45
231,2,xgboost-3,6,ma_num_ba,1.5,6,0.54,109.0,0.56,0.60,0.34,4.79,0.12,0.59,0.54,0.83,0.60,0.59,0.44
327,2,xgboost-3,5,ma_num+pr10,1.5-i,8,0.55,111.0,0.54,0.56,0.35,3.59,0.09,0.59,0.50,0.76,0.56,0.56,0.42
181,2,xgboost-3,6,experimental_slope_ba,1.5,100,0.54,111.0,0.53,0.54,0.35,4.47,0.05,0.55,0.50,0.81,0.59,0.50,0.40
94,2,xgboost-3,4,experimental_slope-baseline,1.5,8,0.57,115.0,0.51,0.48,0.40,6.13,0.02,0.57,0.45,0.83,0.55,0.41,0.37
56,2,xgboost-3,6,experimental_slope-baseline,1.5,8,0.55,222.0,0.50,0.48,0.37,3.93,0.00,0.55,0.45,0.81,0.58,0.38,0.37
333,2,xgboost-3,5,ma_num+exp,1.5-r,8,0.54,107.0,0.50,0.51,0.41,4.77,-0.01,0.54,0.45,0.84,0.48,0.54,0.36
180,2,xgboost-3,4,experimental_slope_ba,1.5,100,0.56,114.0,0.50,0.49,0.40,6.20,-0.00,0.56,0.44,0.84,0.56,0.42,0.36
